<a href="https://colab.research.google.com/github/Agalea96/lightgbm/blob/main/lightgbm_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pickle import TRUE
# importing all needed libraries
import numpy as np
import pandas as pd
import lightgbm
import sklearn.model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# input dataset
df_train = pd.read_csv('/content/Traffic_Violation_Clean.csv', low_memory=False)
def enc_labels(label):
    return label_encoder.fit_transform(df_train[label])
df_train.drop(columns=['Year', 'Year Stopped', 'DateTime Of Stop', 'Geolocation'])
cat_features=df_train.select_dtypes(include=['object', 'int32']).columns
label_encoder = LabelEncoder()

for x in cat_features:
    df_train['encoded_' + x] = enc_labels(x)
filtered_columns=df_train.select_dtypes(include=['bool','int32','int64','float64'])

# selecting all columns that are not the 'ID code' and 'target' columns
var_columns = [c for c in filtered_columns.columns if c not in ['encoded_Violation Type']]

# defining inputs as X labels as Y
X = filtered_columns.loc[:,var_columns]
Y = filtered_columns.loc[:,'encoded_Violation Type']

# splitting training data and labels into training data and labels and validation data and labels
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

# setting up training and validation data so that they can be read by the lightgbm module
train_data = lightgbm.Dataset(X_train, label = Y_train)


In [ ]:
n_splits = 5

kf = sklearn.model_selection.KFold(n_splits=n_splits, shuffle=True, random_state=42)
metrics = []

In [ ]:
# set up parameters
model_lgbm = lightgbm.LGBMClassifier(boosting_type='dart',
                                     objective='multiclass',
#parameters to increase accuracy
                                     learning_rate=0.05,
                                     num_iterations=200,
#parameters to increase accuracy. WARNING, MIGHT CAUSE OVERFITTING
                                     num_leaves=31,
                                     max_bin=255,
#parameters to reduce the chance of overfitting
                                     min_child_samples=20,
                                     min_child_weight=0.001,
                                     subsample=1.0,
                                     subsample_freq=0,
                                     colsample_bytree=1.0,
                                     extra_trees=True,
                                     path_smooth=0,
                                     max_depth=-1,

                                     n_estimators=100,
                                     subsample_for_bin=200000,
                                     class_weight=None,
                                     min_split_gain=0.0,
                                     reg_alpha=0.0,
                                     reg_lambda=0.0,
                                     random_state=None,
                                     n_jobs=None,
                                     importance_type='split',
                                     force_col_wise=True)

In [ ]:
# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

In [ ]:
# fit predictions to training, validation and test data
Y_train_pred = model_lgbm.fit(X_train,Y_train)
Y_test_pred = model_lgbm.fit(X_test,Y_test)

# run predictions on training, validation and test models
Y_train_pred = model_lgbm.predict(X_train, raw_score=False, start_iteration=0, num_iteration=None, pred_leaf=False, pred_contrib=False, validate_features=False)
Y_test_pred = model_lgbm.predict(X_test, raw_score=False, start_iteration=0, num_iteration=None, pred_leaf=False, pred_contrib=False, validate_features=False)



/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 1670
[LightGBM] [Info] Number of data points in the train set: 732395, number of used features: 38
[LightGBM] [Info] Start training from score -0.946695
[LightGBM] [Info] Start training from score -3.321518
[LightGBM] [Info] Start training from score -8.092429
[LightGBM] [Info] Start training from score -0.552386


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 1660
[LightGBM] [Info] Number of data points in the train set: 183098, number of used features: 37
[LightGBM] [Info] Start training from score -0.938117
[LightGBM] [Info] Start training from score -3.324013
[LightGBM] [Info] Start training from score -8.147485
[LightGBM] [Info] Start training from score -0.558025


In [ ]:
# Generate classification reports for training, validation and test results
report = classification_report(Y_train_pred,Y_train)
print('Classification report (Training set):\n', report)

#report = classification_report(Y_valid_pred,Y_valid)
print('Classification report (Validation set):\n', report)

#report = classification_report(Y_test_pred,Y_test)
print('Classification report (Test set):\n', report)

Classification report (Training set):
               precision    recall  f1-score   support

           0       0.81      0.86      0.84    267580
           1       0.78      0.91      0.84     22622
           2       0.01      0.05      0.01        44
           3       0.91      0.86      0.89    442149

    accuracy                           0.86    732395
   macro avg       0.63      0.67      0.64    732395
weighted avg       0.87      0.86      0.87    732395

Classification report (Validation set):
               precision    recall  f1-score   support

           0       0.81      0.86      0.84    267580
           1       0.78      0.91      0.84     22622
           2       0.01      0.05      0.01        44
           3       0.91      0.86      0.89    442149

    accuracy                           0.86    732395
   macro avg       0.63      0.67      0.64    732395
weighted avg       0.87      0.86      0.87    732395

Classification report (Test set):
               p